In [2]:
import pandas as pd


In [3]:
dataset_path = r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset"


In [4]:
customers = pd.read_csv(dataset_path + '/customers.csv')
order_items = pd.read_csv(dataset_path +'/order_items.csv')
order_payments = pd.read_csv(dataset_path +'/order_payments.csv')
order_reviews = pd.read_csv(dataset_path +'/order_reviews.csv')
orders = pd.read_csv(dataset_path +'/orders.csv')
products = pd.read_csv(dataset_path +'/products.csv')
sellers = pd.read_csv(dataset_path +'/sellers.csv')
data = {'customers': customers,
        'order_items': order_items, 
        'order_payments': order_payments,
        'order_reviews': order_reviews,
        'orders': orders,
        'products': products,
        'sellers': sellers}

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\PycharmProjects\\lufthansa_task\\lufthansa_task\\cleaned_dataset/customers.csv'

1. Total Sales per Customer: A running total of product price for each customer partitioned by Customer ID.

In [20]:
# Merge order_items with orders and customers to get the customer_id and customer_unique_id. 
order_items = order_items.merge(orders[['order_id', 'customer_id']], on='order_id', how='left')
order_items = order_items.merge(customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

In [21]:
# Get total sales per customer
order_items['total_sales_per_customer'] = order_items.groupby('customer_unique_id')['price'].cumsum()
order_items[['order_id', 'customer_unique_id', 'total_sales_per_customer']].head()


,order_id,customer_unique_id,total_sales_per_customer
0,00010242fe8c5a6d1ba2dd792cb16214,871766c5855e863f6eccc05f988b23cb,58.90
1,00018f77f2f0320c557190d7a144bdd3,eb28e67c4c0b83846050ddfb8a35d051,239.90
2,000229ec398224ef6ca0657da4fc703e,3818d81c6709e39d06b2738a8d3a2474,199.00
3,00024acbcdf0a6daa1e931b038114c75,af861d436cfc08b2c2ddefd0ba074622,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,64b576fb70d441e8f1b2d7d446e483c5,199.90


2. Average Delivery Time per Product Category: Rolling average delivery time by product category


In [22]:
print(order_items.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110189 entries, 0 to 110188
Data columns (total 16 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       110189 non-null  object 
 1   order_item_id                  110189 non-null  int64  
 2   product_id                     110189 non-null  object 
 3   seller_id                      110189 non-null  object 
 4   shipping_limit_date            110189 non-null  object 
 5   price                          110189 non-null  float64
 6   freight_value                  110189 non-null  float64
 7   total_price                    110189 non-null  float64
 8   order_purchase_timestamp       110189 non-null  object 
 9   order_delivered_customer_date  110189 non-null  object 
 10  delivery_time                  110189 non-null  int64  
 11  total_payments_count           110186 non-null  float64
 12  profit_margin                 

In [23]:
# Merge order_items with products to get the product_category_name.
order_items = order_items.merge(products[['product_id', 'product_category_name']], on='product_id', how='left')

In [24]:
# Sort the values based on order_purchase_timestamp
order_items = order_items.sort_values(by=['product_category_name', 'order_purchase_timestamp'])

In [25]:
# Get the rolling average delivery time by product category
order_items['rolling_avg_delivery_time'] = order_items.groupby('product_category_name')['delivery_time'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)

order_items[['product_category_name', 'delivery_time', 'rolling_avg_delivery_time']].head(100)

,product_category_name,delivery_time,rolling_avg_delivery_time
4364,Other,18,18.000000
77935,Other,19,18.500000
82729,Other,7,14.666667
42153,Other,13,13.000000
14701,Other,8,9.333333
...,...,...,...
87033,Other,15,14.000000
48364,Other,4,11.333333
96612,Other,15,11.333333
28568,Other,25,14.666667


In [26]:
order_items.to_csv(r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset\order_items.csv", index=False)

In [27]:
order_items[['order_id', 'order_item_id', 'total_payments_count']].head(10)

,order_id,order_item_id,total_payments_count
4364,0a0837a5eee9e7a9ce2b1fa831944d27,1,1.0
77935,b533dea24ada4dc5c1a9041d25d7c812,1,1.0
82729,c022b7da8ab7a4a04bd561c8f765a14a,1,1.0
42153,6219ece1f053a3ffa649340e87cd93d2,1,3.0
14701,22474d84064cd7eb4a907ba3fd8a6398,1,4.0
...,...,...,...
87033,ca1533a4041a98501d0c3250c7dff345,2,1.0
48364,707109d106f57a298c6d726ff59fa324,1,10.0
96612,dffe3c3fdb38dd7ca56eb98408730378,1,1.0
28568,427be8c69327b86f12e65fb5fedfde5c,1,1.0
